In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import csv
from enum import Enum

from loguru import logger
from pydantic import BaseModel

from taskllm.optimizer.data import DataSet, Row
from taskllm.optimizer.methods import BanditTrainer, BayesianTrainer
from taskllm.optimizer.prompt.meta import PromptMode

# logger.remove()  # remove the old handler. Else, the old one will work (and continue printing DEBUG logs) along with the new handler added below'
# logger.add(sys.stdout, level="TRACE")  # add a new handler which has INFO as the default


class Ratings(Enum):
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    NA = "N/A"


class StarbucksReviewRating(BaseModel):
    rating: Ratings


def sentiment_scoring_function(
    row: Row[StarbucksReviewRating], output: StarbucksReviewRating | None
) -> float:
    if output is None:
        return -10
    if not row.expected_output:
        return 0
    logger.trace(f"Expected: {row.expected_output.rating}, Output: {output.rating}")
    if row.expected_output.rating == output.rating:
        return 1

    return 0


def load_file_as_dataset(path: str) -> DataSet:
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            rating = StarbucksReviewRating(rating=Ratings(row["Rating"]))
            rows.append(
                Row.create(
                    input_dictionary={
                        "review": row["Review"],
                        "name": row["name"],
                        "location": row["location"],
                        "date": row["Date"],
                    },
                    output=rating,
                )
            )
    return DataSet(rows=rows[:50], name="starbucks_reviews")



/Users/bill/dev/taskllm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
csv_path = "./starbucks_reviews.csv"
dataset = load_file_as_dataset(csv_path)
trainer = BayesianTrainer(
    all_rows=dataset,
    task_guidance="determine the rating of this review",
    keys=["review", "name", "location", "date"],
    expected_output_type=StarbucksReviewRating,
    scoring_function=sentiment_scoring_function,
    num_iterations=2,  # Start with fewer iterations for testing
    candidates_per_iteration=2,  # Start with fewer candidates for testing
    prompt_mode=PromptMode.SIMPLE,
    models=[
        "anthropic/claude-3-haiku-20240307",
        "openai/gpt-4.1-nano-2025-04-14",
        "openai/gpt-4.1-mini-2025-04-14",
        "groq/gemma2-9b-it",
        "groq/qwen-qwq-32b"
    ],
)

2025-05-12 20:57:11.870 | INFO     | taskllm.optimizer.methods.bayesian:__init__:208 - Using CPU for Pyro/Torch computations
2025-05-12 20:57:11.871 | INFO     | taskllm.optimizer.methods.base:__init__:242 - All rows: 50
2025-05-12 20:57:11.871 | DEBUG    | taskllm.optimizer.methods.bayesian:__init__:721 - BayesianTrainer initialized


In [4]:
await trainer.train()

2025-05-12 20:57:11.899 | INFO     | taskllm.optimizer.methods.bayesian:train:725 - Starting Bayesian optimization with Pyro: 2 iterations, 2 candidates/iter.
2025-05-12 20:57:11.899 | INFO     | taskllm.optimizer.methods.bayesian:train:731 - Phase 1: Generating and evaluating initial candidates...
2025-05-12 20:57:11.900 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:139 - Generating simple prompt content for: determine the rating of this review

Use plain language in the prompt you write...
2025-05-12 20:57:14.950 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:139 - Generating simple prompt content for: determine the rating of this review

Make the prompt you write as simple as possible...


You are a review evaluator. Your task is to analyze the review provided and assign a star rating based on its content. Take into account the tone, details, and overall sentiment expressed in the review.

Here are the details of the review:
- Review: {{ review }}
- Name of the reviewer: {{ name }}
- Location of the reviewer: {{ location }}
- Date of the review: {{ date }}

After your analysis, please clearly state the star rating from 1 to 5, with 1 being poor and 5 being excellent.


2025-05-12 20:57:16.321 | INFO     | taskllm.optimizer.methods.bayesian:train:754 - Generated 2 initial candidate prompts.
2025-05-12 20:57:16.323 | INFO     | taskllm.optimizer.methods.base:run_for_prompt:280 - Running for prompt You are a review evaluator. Your task is to analyze the review provided and assign a star rating bas... against 40 rows
2025-05-12 20:57:16.325 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4.1-nano-2025-04-14
2025-05-12 20:57:16.326 | INFO     | taskllm.optimizer.methods.base:run_for_prompt:280 - Running for prompt You are a review rating expert. Analyze the review provided by the user and assign it a rating betwe... against 40 rows
2025-05-12 20:57:16.326 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4.1-nano-2025-04-14


You are a review rating expert. Analyze the review provided by the user and assign it a rating between 1 and 5 based on its content. Consider the tone, content quality, and overall impression. The review details are as follows: Review: {{ review }}; Name: {{ name }}; Location: {{ location }}; Date: {{ date }}.


2025-05-12 20:57:56.674 | INFO     | taskllm.optimizer.methods.bayesian:train:766 - Fitting initial surrogate model
2025-05-12 20:57:56.675 | SUCCESS  | taskllm.optimizer.methods.bayesian:fit_surrogate_model:267 - Extracted features for 2 prompts
2025-05-12 20:57:56.676 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4.1-nano-2025-04-14
2025-05-12 20:57:56.677 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4.1-nano-2025-04-14
2025-05-12 20:57:56.714 | DEBUG    | taskllm.optimizer.methods.base:get_scores:72 - Calculated 40 scores
2025-05-12 20:57:56.715 | INFO     | taskllm.optimizer.methods.base:calculate_scores:97 - Model openai/gpt-4.1-nano-2025-04-14 with prompt achieved score: 31.0000, Correct: 31, Incorrect: 9, Unlabelled: 0 out of 40
2025-05-12 20:57:56.715 | DEBUG    | taskllm.optimizer.methods.base:get_scores:72 - Calculated 40 scores
2025-05-12 20:57:56.715 | IN

You are a review analysis expert. Read the review provided below and determine its rating on a scale from 1 to 5. 

Review: {{ review }}
Reviewer Name: {{ name }}
Location: {{ location }}
Date: {{ date }}

Provide a clear rating based on the content and tone of the review.


2025-05-12 20:58:01.322 | INFO     | taskllm.optimizer.methods.bayesian:train:802 - Generated 2 new candidates for evaluation.
2025-05-12 20:58:01.323 | INFO     | taskllm.optimizer.methods.base:run_for_prompt:280 - Running for prompt You are a review analysis expert. Read the review provided below and determine its rating on a scale... against 40 rows
2025-05-12 20:58:01.324 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4o-mini
2025-05-12 20:58:01.326 | INFO     | taskllm.optimizer.methods.base:run_for_prompt:280 - Running for prompt As a rating assessor, read the review provided and evaluate its sentiment. Based on the content, det... against 40 rows
2025-05-12 20:58:01.326 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4o-mini


As a rating assessor, read the review provided and evaluate its sentiment. Based on the content, determine a rating from 1 to 5, where 1 is the lowest and 5 is the highest. Provide your rating along with a brief explanation of your reasoning.

Review: {{ review }}
Reviewer Name: {{ name }}
Location: {{ location }}
Date of Review: {{ date }}


2025-05-12 20:58:36.318 | INFO     | taskllm.optimizer.methods.bayesian:train:813 - Prompt content: You are a review analysis expert. Read the review provided below and determine its rating on a scale...
2025-05-12 20:58:36.319 | INFO     | taskllm.optimizer.methods.bayesian:train:813 - Prompt content: As a rating assessor, read the review provided and evaluate its sentiment. Based on the content, det...
2025-05-12 20:58:36.320 | INFO     | taskllm.optimizer.methods.bayesian:train:818 - Updating surrogate model with new data
2025-05-12 20:58:36.321 | SUCCESS  | taskllm.optimizer.methods.bayesian:fit_surrogate_model:267 - Extracted features for 4 prompts
2025-05-12 20:58:36.322 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4.1-nano-2025-04-14
2025-05-12 20:58:36.322 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:55 - Getting outputs for 40 rows using openai/gpt-4.1-nano-2025-04-14
2025-05-12 20:58:36.323 | DEBUG    | t